# Data Preprocessing

In [22]:
import pandas as pd
import numpy as np
import re # Regular expression library
import string


data = pd.read_csv('Hotel Reservations.csv')
df = pd.DataFrame(data)
df.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


In [54]:


# Correcting the 'lower' lambda function
lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower()) 

# Applying the functions to the DataFrame
# Applying lowercase to all string columns in the DataFrame
for col in df.columns:
    if df[col].dtype == 'object':  # typically string columns are of 'object' type
        df[col] = df[col].map(lower)
        
df.head() 

#df.info()
#df.isnull().sum()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,2,meal plan 1,0,room type 1,224,2017,10,2,offline,0,0,0,65.00,0,not canceled
1,2,0,2,3,not selected,0,room type 1,5,2018,11,6,online,0,0,0,106.68,1,not canceled
2,1,0,2,1,meal plan 1,0,room type 1,1,2018,2,28,online,0,0,0,60.00,0,canceled
3,2,0,0,2,meal plan 1,0,room type 1,211,2018,5,20,online,0,0,0,100.00,0,canceled
4,2,0,1,1,not selected,0,room type 1,48,2018,4,11,online,0,0,0,94.50,0,canceled


# Split the data for train and test data

In [50]:
features = df[['no_of_adults', 'no_of_children','no_of_weekend_nights','no_of_week_nights', 'type_of_meal_plan', 	
               'required_car_parking_space', 'room_type_reserved', 'lead_time', 'arrival_year', 'arrival_month', 	
               'arrival_date', 'market_segment_type', 'repeated_guest', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled', 	
               'avg_price_per_room', 'no_of_special_requests']]

label = df['booking_status']


0    not canceled
1    not canceled
2        canceled
3        canceled
4        canceled
Name: booking_status, dtype: object

In [56]:
from sklearn.model_selection import train_test_split
features_train, features_test, label_train, label_test = train_test_split(features, label, test_size=0.3, random_state=42)


,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
29580,2,0,1,3,meal plan 1,0,room type 1,200,2018,8,29,online,0,0,0,90.95,0
6325,2,0,1,3,meal plan 1,0,room type 1,79,2018,3,24,online,0,0,0,90.95,1
33899,2,0,1,4,meal plan 1,0,room type 4,78,2018,4,6,online,0,0,0,99.45,1
720,2,0,2,0,meal plan 1,0,room type 1,61,2017,10,4,offline,0,0,0,91.00,0
22120,2,1,0,4,meal plan 1,0,room type 1,201,2018,11,1,online,0,0,0,82.28,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16850,2,0,2,2,meal plan 1,0,room type 1,43,2017,12,26,offline,0,0,0,55.00,2
6265,1,0,0,2,meal plan 2,0,room type 1,102,2017,10,16,online,0,0,0,80.00,0
11284,2,0,0,2,meal plan 1,0,room type 1,5,2018,5,24,corporate,0,0,0,110.00,1
860,2,0,0,3,meal plan 1,0,room type 1,213,2018,6,7,online,0,0,0,130.00,0
